<a href="https://colab.research.google.com/github/ywjin0707/bioinformatics-playground/blob/main/SequenceAlignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence Alignment
This is my attempt to make an algorithm to align short oligonucleotide code onto a short reference genome after reading the [STAR paper by Dobin et al. 2013](https://dx.doi.org/10.1093%2Fbioinformatics%2Fbts635).  
After reading, I came to the realization that the 'uncompressed suffix array' the authors talked about were much like n-grams if the code is treated as a string. A little differently from the STAR alignment strategy, I decided to find the longest matching n-gram between the query and reference and use that as anchor to match ***both the prefix and the suffix***.  

In [ ]:
# !pip install python-levenshtein

In [ ]:
from nltk.util import *
# from Levenshtein import *

In [ ]:
reference = 'ACTGGGTACGTAGCGTAGGAGACGAATCGATCGCTCTTCTTTCCTAGCAAATCGACCTAACGCATACGCA'
query = 'ACGCTAGCTAGCTCAGACTAG' # Handle one query for now for pairwise alignment. 

In [ ]:
grams_ref = list(everygrams(reference))

In [ ]:
grams_query = list(everygrams(query))

In [ ]:
grams_query.sort(key=len, reverse=True) # Sort to search for longest match first

In [ ]:
## Find Maximal Mappable Prefixes (MMPs)
for gram in grams_query:
  if gram in grams_ref:
    seed = ''.join(gram)
    idx_ref = reference.index(seed) # Currently only finds the first index. In the future repeat and multiple mapping should be considered
    idx_query = query.index(seed)
    # print(seed)
    # print(idx)
    # print(reference[idx:idx+len(seed)])
    break


In [ ]:
split_query = query.split(seed) # change to index-based slicing later
if len(split_query) > 1:
  prefix = split_query[0]
suffix = split_query[-1]

In [ ]:
prefix

'ACG'

In [ ]:
suffix

'TAGCTCAGACTAG'

In [ ]:
# Use recursion to solve
def findAlignment(ref, query, refidx=0):
  print('=================================================================')
  if len(ref) == 0:
    print('ref length 0!')
    return
  global reference
  global alignedSeq
  global currentLen
  print('original reference seq: ', reference)
  print('current reference seq: ', ref)
  print('query seq: ', query)

  # max_seed_Len = -1 if max_seed_L > len(ref) else max_seed_L # adjust to length of ref sequence
  grams_ref = list(everygrams(ref))#, min_len=min_seed_L, max_len=max_seed_L))
  # max_seed_Len = -1 if max_seed_L > len(query) else max_seed_L # adjust to length of query sequence
  grams_query = list(everygrams(query))#, min_len=min_seed_L, max_len=max_seed_Len))
  # Sort to search for longest match first
  grams_query.sort(key=len, reverse=True)
  ## Initialize values
  seed = ''
  idx_ref = 0
  idx_query = len(query)
  ## Find Maximal Mappable Prefixes (MMPs)
  for gram in grams_query:
    if gram in grams_ref:
      seed = ''.join(gram)
      idx_ref = ref.index(seed)
      idx_query = query.index(seed)
      break
  idx_reference = refidx + idx_ref # Currently only finds the first index. In the future repeat and multiple mapping should be considered
  ## For debugging
  print('Seed: ', seed)
  print('Start index: ', idx_reference, '   End index: ', idx_reference+len(seed))
  # print(reference[idx_reference:idx_reference+len(seed)])
  if len(seed) > 0:
    alignedSeq.append({seed:{'idx_start':idx_reference, 'idx_end':idx_reference+len(seed)}})
    # if currentLen < len(query):
    prefix = query[:idx_query]
    suffix = query[idx_query+len(seed):]
    print('prefix: ', prefix)
    print('suffix: ', suffix)
    if len(prefix) > 1 & len(prefix) < len(ref[:idx_ref]):
      findAlignment(ref[:idx_ref], prefix, refidx=refidx)
    elif len(prefix) == 1:
      alignedSeq.append({prefix:{'idx_start': refidx, 'idx_end': refidx+len(prefix)}})
    if len(suffix) > 1 & len(suffix) < len(ref[idx_ref+len(seed):]):
      findAlignment(ref[idx_ref+len(seed):], suffix, refidx=refidx+idx_ref+len(seed))
    elif len(suffix) == 1:
      alignedSeq.append({suffix:{'idx_start': refidx, 'idx_end': refidx+len(suffix)}})
  else:
    alignedSeq.append({query:{'idx_start': idx_reference, 'idx_end': idx_reference+len(query)}})


In [ ]:
alignedSeq = []
currentLen = 0
findAlignment(reference, query)

original reference seq:  ACTGGGTACGTAGCGTAGGAGACGAATCGATCGCTCTTCTTTCCTAGCAAATCGACCTAACGCATACGCA
current reference seq:  ACTGGGTACGTAGCGTAGGAGACGAATCGATCGCTCTTCTTTCCTAGCAAATCGACCTAACGCATACGCA
query seq:  ACGCTAGCTAGCTCAGACTAG
Seed:  CTAGC
Start index:  43    End index:  48
prefix:  ACG
suffix:  TAGCTCAGACTAG
original reference seq:  ACTGGGTACGTAGCGTAGGAGACGAATCGATCGCTCTTCTTTCCTAGCAAATCGACCTAACGCATACGCA
current reference seq:  ACTGGGTACGTAGCGTAGGAGACGAATCGATCGCTCTTCTTTC
query seq:  ACG
Seed:  ACG
Start index:  7    End index:  10
prefix:  
suffix:  
original reference seq:  ACTGGGTACGTAGCGTAGGAGACGAATCGATCGCTCTTCTTTCCTAGCAAATCGACCTAACGCATACGCA
current reference seq:  AAATCGACCTAACGCATACGCA
query seq:  TAGCTCAGACTAG
Seed:  GAC
Start index:  53    End index:  56
prefix:  TAGCTCA
suffix:  TAG
original reference seq:  ACTGGGTACGTAGCGTAGGAGACGAATCGATCGCTCTTCTTTCCTAGCAAATCGACCTAACGCATACGCA
current reference seq:  AAATC
query seq:  TAGCTCA
Seed:  TC
Start index:  51    End index:  53
prefix:  T

In [ ]:
alignedSeq

[{'CTAGC': {'idx_end': 48, 'idx_start': 43}},
 {'ACG': {'idx_end': 10, 'idx_start': 7}},
 {'GAC': {'idx_end': 56, 'idx_start': 53}},
 {'TC': {'idx_end': 53, 'idx_start': 51}},
 {'A': {'idx_end': 49, 'idx_start': 48}},
 {'T': {'idx_end': 49, 'idx_start': 48}},
 {'GC': {'idx_end': 51, 'idx_start': 49}},
 {'A': {'idx_end': 49, 'idx_start': 48}},
 {'TA': {'idx_end': 59, 'idx_start': 57}},
 {'G': {'idx_end': 57, 'idx_start': 56}}]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'reference':[s for s in reference],
                   'query':['-']*len(reference)})

In [ ]:
for seed_dict in alignedSeq:
  for seed, idxs in seed_dict.items():
    df.iloc[idxs['idx_start']:idxs['idx_end'],1] = [s for s in seed]

In [ ]:
def findMismatch(row):
  if row['query'] == '-':
    return ' '
  elif row['query'] == row['reference']:
    return ' '
  else:
    return 'X'

In [ ]:
df['mismatch'] = df.apply(lambda row: findMismatch(row), axis=1)
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69
reference,A,C,T,G,G,G,T,A,C,G,T,A,G,C,G,T,A,G,G,A,G,A,C,G,A,A,T,C,G,A,T,C,G,C,T,C,T,T,C,T,T,T,C,C,T,A,G,C,A,A,A,T,C,G,A,C,C,T,A,A,C,G,C,A,T,A,C,G,C,A
query,-,-,-,-,-,-,-,A,C,G,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,C,T,A,G,C,A,G,C,T,C,G,A,C,G,T,A,-,-,-,-,-,-,-,-,-,-,-
mismatch,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,X,X,,,,,,X,,,,,,,,,,,,,
